In [1]:
import kagglehub

path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

100%|██████████| 787M/787M [00:07<00:00, 104MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1


In [2]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from PIL import UnidentifiedImageError

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
EPOCHS = 5
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2

DATASET_PATH = os.path.join(path, 'kagglecatsanddogs_3367a', 'PetImages')
SPLIT_DATASET_PATH = "PetImages_Split"
CLASSES = ["Cat", "Dog"]

# Function to clean and split the dataset
def clean_and_split_dataset(dataset_path, output_path, train_ratio=0.7, val_ratio=0.2):
    # Create train/val/test directories
    for subset in ["train", "val", "test"]:
        for cls in CLASSES:
            os.makedirs(os.path.join(output_path, subset, cls), exist_ok=True)

    # Process and split each class
    for cls in CLASSES:
        cls_path = os.path.join(dataset_path, cls)
        images = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # Check and clean corrupt images
        valid_images = []
        for img in images:
            img_path = os.path.join(cls_path, img)
            try:
                tf.keras.preprocessing.image.load_img(img_path).close()
                valid_images.append(img)
            except (UnidentifiedImageError, OSError):
                os.remove(img_path)
                print(f"Removed corrupt image: {img_path}")

        # Split into train, val, and test
        train, temp = train_test_split(valid_images, train_size=train_ratio, random_state=42)
        val, test = train_test_split(temp, test_size=(1 - train_ratio - val_ratio) / (1 - train_ratio), random_state=42)

        # Move files into respective folders
        for subset, split in zip(["train", "val", "test"], [train, val, test]):
            for img in split:
                src = os.path.join(cls_path, img)
                dst = os.path.join(output_path, subset, cls, img)
                shutil.copy(src, dst)

    print("Dataset cleaned and split into train/val/test subsets.")

# Clean and split the dataset
clean_and_split_dataset(DATASET_PATH, SPLIT_DATASET_PATH)

# Data generators
data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training and validation datasets
train_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "train"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

validation_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "val"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Test dataset
test_generator = data_gen.flow_from_directory(
    os.path.join(SPLIT_DATASET_PATH, "test"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Dataset cleaned and split into train/val/test subsets.
Found 17471 images belonging to 2 classes.
Found 4990 images belonging to 2 classes.
Found 2498 images belonging to 2 classes.


**Training Base Model (MobileNet) on "Cat vs Dog" dataset and checking results**

In [3]:
# Function to create MobileNet model for retraining
def create_model():
    # Initialize MobileNet with random weights
    base_mobilenet = tf.keras.applications.MobileNet(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        # No pre-trained weights
        weights=None
    )
    model = Sequential([
        base_mobilenet,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        # Binary classification output
        Dense(1, activation="sigmoid")
    ])
    return model

# Compile and train model
model = create_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

print("Retraining the MobileNet model from scratch...")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)

# Evaluate model
loss, accuracy = model.evaluate(test_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Retraining the MobileNet model from scratch...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


137/137 ━━━━━━━━━━━━━━━━━━━━ 159s 791ms/step - accuracy: 0.5734 - loss: 0.7367 - val_accuracy: 0.4996 - val_loss: 0.7699
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 133s 563ms/step - accuracy: 0.6695 - loss: 0.6139 - val_accuracy: 0.4996 - val_loss: 0.8903
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 72s 507ms/step - accuracy: 0.7422 - loss: 0.5208 - val_accuracy: 0.4996 - val_loss: 1.0570
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 510ms/step - accuracy: 0.7965 - loss: 0.4385 - val_accuracy: 0.5351 - val_loss: 0.6811
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 72s 513ms/step - accuracy: 0.8372 - loss: 0.3678 - val_accuracy: 0.7677 - val_loss: 0.4839
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 704ms/step - accuracy: 0.7687 - loss: 0.4815
Validation Loss: 0.4835
Validation Accuracy: 0.7666


**Checking results after performing Transfer Learning**

In [4]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

def create_transfer_learning_model():
    # Load the MobileNet base model
    base_mobilenet = MobileNet(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    # Freeze the base layers
    base_mobilenet.trainable = False

    # Create a new model by adding custom layers on top of the pre-trained base
    model = Sequential([
        base_mobilenet,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile and train transfer learning model
transfer_learning_model = create_transfer_learning_model()
transfer_learning_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Training Transfer Learning Model (MobileNet with ImageNet weights)...")
history_transfer = transfer_learning_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)


transfer_learning_model.save('transfer_learning_mobilenet_model.keras')

#MODEL EVALUATION

transfer_learning_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
loss, accuracy = transfer_learning_model.evaluate(test_generator)


print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Transfer Learning Model (MobileNet with ImageNet weights)...
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 449ms/step - accuracy: 0.8316 - loss: 0.3592 - val_accuracy: 0.9808 - val_loss: 0.0619
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 67s 476ms/step - accuracy: 0.9660 - loss: 0.0902 - val_accuracy: 0.9838 - val_loss: 0.0472
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 59s 415ms/step - accuracy: 0.9727 - loss: 0.0748 - val_accuracy: 0.9862 - val_loss: 0.0413
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 89s 468ms/step - accuracy: 0.9767 - loss: 0.0638 - val_accuracy: 0.9846 - val_loss: 0.0406
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 59s 416ms/step - accuracy: 0.9777 - loss: 0.0581 - val_accuracy: 0.9864 - val_loss: 0.0362
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 331ms/step - accuracy: 0.9846 - loss: 0.0361
Test Loss: 0.0367
Test Accuracy: 0.9856


In [ ]:
# Use the following base models to evaluate the performance on the "Cat vs Dog" dataset
# LeNet
# AlexNet
# VGGNet
# ResNet

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_lenet_model():
    model = Sequential([
        Conv2D(6, kernel_size=(5,5), activation='tanh', input_shape=(*IMAGE_SIZE, 3), padding='same'),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(16, kernel_size=(5,5), activation='tanh'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(120, activation='tanh'),
        Dense(84, activation='tanh'),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

lenet_model = create_lenet_model()
lenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lenet_model.fit(train_generator, validation_data=validation_generator, epochs=5)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 76s 492ms/step - accuracy: 0.5671 - loss: 0.7543 - val_accuracy: 0.6281 - val_loss: 0.6451
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 64s 405ms/step - accuracy: 0.6515 - loss: 0.6237 - val_accuracy: 0.6349 - val_loss: 0.6343
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 413ms/step - accuracy: 0.6550 - loss: 0.6138 - val_accuracy: 0.6553 - val_loss: 0.6100
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 57s 404ms/step - accuracy: 0.6816 - loss: 0.5828 - val_accuracy: 0.6625 - val_loss: 0.6156
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 92s 476ms/step - accuracy: 0.7087 - loss: 0.5634 - val_accuracy: 0.6908 - val_loss: 0.5772


In [6]:
def create_alexnet_model():
    model = Sequential([
        Conv2D(96, kernel_size=(11,11), strides=4, activation='relu', input_shape=(*IMAGE_SIZE, 3)),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Conv2D(256, kernel_size=(5,5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Conv2D(384, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

alexnet_model = create_alexnet_model()
alexnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
alexnet_model.fit(train_generator, validation_data=validation_generator, epochs=5)


Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 100s 579ms/step - accuracy: 0.4974 - loss: 0.8372 - val_accuracy: 0.5004 - val_loss: 0.6931
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 104s 424ms/step - accuracy: 0.4927 - loss: 0.6933 - val_accuracy: 0.4996 - val_loss: 0.6932
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 81s 423ms/step - accuracy: 0.4969 - loss: 0.6932 - val_accuracy: 0.5004 - val_loss: 0.6931
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 425ms/step - accuracy: 0.5103 - loss: 0.6932 - val_accuracy: 0.5004 - val_loss: 0.6932
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 81s 420ms/step - accuracy: 0.4920 - loss: 0.6934 - val_accuracy: 0.5004 - val_loss: 0.6931


In [7]:
from tensorflow.keras.applications import VGG16

def create_vgg_model():
    base_vgg = VGG16(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    base_vgg.trainable = False
    model = Sequential([
        base_vgg,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

vgg_model = create_vgg_model()
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
vgg_model.fit(train_generator, validation_data=validation_generator, epochs=5)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.8102 - loss: 0.3998 - val_accuracy: 0.9477 - val_loss: 0.1425
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 110s 786ms/step - accuracy: 0.9423 - loss: 0.1451 - val_accuracy: 0.9593 - val_loss: 0.1124
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 110s 781ms/step - accuracy: 0.9554 - loss: 0.1139 - val_accuracy: 0.9525 - val_loss: 0.1210
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 158s 896ms/step - accuracy: 0.9551 - loss: 0.1128 - val_accuracy: 0.9607 - val_loss: 0.1034
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 125s 774ms/step - accuracy: 0.9579 - loss: 0.1064 - val_accuracy: 0.9617 - val_loss: 0.0995


In [8]:
from tensorflow.keras.applications import ResNet50

def create_resnet_model():
    base_resnet = ResNet50(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    base_resnet.trainable = False
    model = Sequential([
        base_resnet,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

resnet_model = create_resnet_model()
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_generator, validation_data=validation_generator, epochs=5)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 120s 710ms/step - accuracy: 0.5892 - loss: 0.7007 - val_accuracy: 0.6928 - val_loss: 0.6038
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 99s 508ms/step - accuracy: 0.6537 - loss: 0.6191 - val_accuracy: 0.7002 - val_loss: 0.5806
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 83s 518ms/step - accuracy: 0.6695 - loss: 0.6035 - val_accuracy: 0.6886 - val_loss: 0.5887
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 77s 548ms/step - accuracy: 0.6964 - loss: 0.5860 - val_accuracy: 0.7040 - val_loss: 0.5700
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 77s 550ms/step - accuracy: 0.6834 - loss: 0.5854 - val_accuracy: 0.7291 - val_loss: 0.5553


In [10]:
# Perform Transfer Learning using the following models and check results
# LeNet
# AlexNet
# VGGNet
# ResNet

In [18]:
# Train and Evaluate Models

def lenet_model(input_shape=(32, 32, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(6, (5, 5), activation='relu')(inputs)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)  # Specify pool_size
    x = tf.keras.layers.Conv2D(16, (5, 5), activation='relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)  # Specify pool_size
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(120, activation='relu')(x)
    x = tf.keras.layers.Dense(84, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# Ensure this is how your vgg_model works
def vgg_model(input_shape=(224, 224, 3)):
    inputs = tf.keras.Input(shape=input_shape)  # Create the input tensor
    # Add the VGG layers here
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    # Additional layers as needed
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)


def train_and_evaluate(model, name):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
    print(f"Training {name}...")
    history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, verbose=1)
    loss, acc = model.evaluate(test_generator, verbose=1)
    print(f"{name} Test Loss: {loss:.4f}")
    print(f"{name} Test Accuracy: {acc:.4f}\n")
    return acc

# Instantiate Models with Correct Input Sizes
lenet = lenet_model(input_shape=(32, 32, 3))
alexnet = alexnet_model(input_shape=(227, 227, 3))
vggnet = vgg_model(input_shape=(224, 224, 3))
resnet = resnet_model(input_shape=(224, 224, 3))

# Train and Evaluate
results = {
    "LeNet": train_and_evaluate(lenet, "LeNet"),
    "AlexNet": train_and_evaluate(alexnet, "AlexNet"),
    "VGGNet": train_and_evaluate(vggnet, "VGGNet"),
    "ResNet": train_and_evaluate(resnet, "ResNet"),
}

# Print Final Results
print("Final Model Performance:")
for model_name, acc in results.items():
    print(f"{model_name}: {acc:.4f}")


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[802816,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 